# Datos de Voz

**Para los datos de voz usamos el DataSet de "The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS )" obtenido en la pagina "https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio"**

Con el fin de utilizarlos de una mejor manera se obtuvieron las etiquetas de las categorias de cada audio como especifica el autor.

Cada uno de los 1440 archivos tiene un nombre de archivo único. El nombre del archivo consta de un identificador numérico de 7 partes (p. ej., 03-01-06-01-02-01-12.wav). Estos identificadores definen las características del estímulo:

Identificadores de nombre de archivo

* Modalidad (01 = full-AV, 02 = solo video, 03 = solo audio).
* Canal vocal (01 = habla, 02 = canción).
* Emoción (01 = neutral, 02 = tranquilo, 03 = feliz, 04 = triste, 05 = enojado, 06 = temeroso, 07 = asco, 08 = sorprendido).
* Intensidad emocional (01 = normal, 02 = fuerte). NOTA: No hay una intensidad fuerte para la emoción 'neutral'.
* Declaración (01 = "Los niños hablan junto a la puerta", 02 = "Los perros están sentados junto a la puerta").
* Repetición (01 = 1ª repetición, 02 = 2ª repetición).
* Actor (del 01 al 24. Los actores impares son hombres, los actores pares son mujeres).

En el siguiente codigo vamos a almacenar los datos en un archivo Json para poder utilizar los espectrogramas sin tener que volver a cargar los audios, usamos scipy.interpolate para tener los datos del espectrograma de un mismo tamaño.


In [ ]:
import os
import json
import librosa
import numpy as np
from scipy.interpolate import interp1d

# Establecer la ruta de la carpeta que contiene los archivos de audio
ruta_carpeta = "archive/"
print(os.listdir(ruta_carpeta)[:-1])
# Crear una lista vacía para almacenar los datos del espectrograma de cada archivo de audio
espectrograma = []
max_length = 240

# Recorrer cada archivo de audio en la carpeta
for carpeta in os.listdir(ruta_carpeta)[:-1]:
    ruta_carpeta_audio = ruta_carpeta+carpeta+"/"
    print(ruta_carpeta_audio)
    for archivo_audio in os.listdir(ruta_carpeta_audio):
        # Comprobar que el archivo es un archivo de audio
        if archivo_audio.endswith(".wav"):
            # Obtener las etiquetas de categoría del archivo de audio a partir del nombre del archivo
            etiquetas = archivo_audio.split("-")
            Modality = etiquetas[0]  # 01-full, 02-Video, 03-audio
            Vocal = etiquetas[1]  # 01-speech, 02-song
            Emotion = etiquetas[2]  # 01-neutral, 02-calm, 03-happy, 04-sad, 05-angry, 06-fearful, 07-disgust, 08-surprised
            Intensity = etiquetas[3]  # 01-normal, 02-strong
            Statement = etiquetas[4]  # 01-"Kids are talking by the door", 02-"Dogs are sitting by the door").
            Repetition = etiquetas[5] # 01-1st repetition, 02-2nd repetition).
            Actor = etiquetas[6].split(".")[0] # Par-Mujer, Impar-Hombre

            # Cargar el archivo de audio y extraer los espectrogramas
            ruta_archivo_audio = os.path.join(ruta_carpeta_audio, archivo_audio)
            y, sr = librosa.load(ruta_archivo_audio,sr=22050)
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            f = interp1d(np.linspace(0, 1, mel_spec_db.shape[1]), mel_spec_db, kind='linear', axis=1)
            mel_spec_db = f(np.linspace(0, 1, max_length))

            # Agregar los datos del espectrograma a la lista junto con las etiquetas de categoría
            espectrograma.append({
                "Modality": Modality,
                "Vocal": Vocal,
                "Emotion": Emotion,
                "Intensity": Intensity,
                "Statement": Statement,
                "Repetition": Repetition,
                "Actor": Actor,
                "Espectrograma": mel_spec_db.tolist()
            })

# Guardar los datos del espectrograma en un archivo JSON
ruta_archivo_json = "Espectrograma.json"
with open(ruta_archivo_json, "w") as archivo_json:
    json.dump(espectrograma, archivo_json)


# Carga de datos

**Cargamos los datos y los guardamos en una variable llamada data**

In [ ]:
import os
import json
# Crear datos de entrenamiento
ruta_archivo_json =  "Espectrograma.json"
with open(ruta_archivo_json, "r") as archivo_json:
    data = json.load(archivo_json)
print(len(data))

# Datos

**Guardamos los datos en un DataFrame con la finalidad de poder usarlos de una manera mas facil**

Tambien hacemos un acomodo aleatorio de los datos y normalizamos los datos del espectrograma

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame()
df["Modality"] = [int(data[val]['Modality'])-1 for val in range(len(data)) ]
df["Vocal"] = [int(data[val]['Vocal'])-1 for val in range(len(data)) ]
df["Emotion"] = [int(data[val]['Emotion'])-1 for val in range(len(data)) ]
df["Intensity"] = [int(data[val]['Intensity'])-1 for val in range(len(data)) ]
df["Statement"] = [int(data[val]['Statement'])-1 for val in range(len(data)) ]
df["Repetition"] = [int(data[val]['Repetition'])-1 for val in range(len(data)) ]
df["Actor"] = [int(data[val]['Actor'])-1 for val in range(len(data)) ]
df["Espectrograma"] = [np.array(data[val]['Espectrograma']) for val in range(len(data)) ]

df=df.sample(frac=1).reset_index(drop=True)
print(df)
df_normalized = df["Espectrograma"].apply(lambda x: (x - x.min()) / (x.max() - x.min()) * 2 - 1)
print(df_normalized)

# Preparacion de datos para entrenamiento

**Acomodamos los datos dejando un 20% para el test y del 80% de los datos resntante un 20% para la validacion, los datos restantes a esto seran los que usaremos para entrenar**

In [ ]:
porcentaje = 0.8
total = len(df["Actor"])
split = porcentaje*total


data_c = np.array([np.asarray(x) for x in df['Espectrograma'][:int(split)]])
data_e = np.asarray(df["Actor"][:int(split)])

porcentaje = 0.8
total = len(data_c)
split2 = porcentaje*total

#Train data 
train_data = data_c[:int(split2)]
train_data = train_data.astype(np.float)
train_data = train_data.reshape(-1, 128, 240, 1)

train_labels = data_e[:int(split2)]
train_labels = np.asarray(train_labels)
train_labels = train_labels.astype(np.float)
train_labels = train_labels.reshape(-1, 1)

print("train_data shape: ",train_data.shape)
print("train_labels shape: ",train_labels.shape)

#Valid data
val_data = data_c[int(split2):]
val_data = val_data.astype(np.float)
val_data = val_data.reshape(-1, 128, 240, 1)
val_labels = data_e[int(split2):]
val_labels = val_labels.astype(np.float)
val_labels = val_labels.reshape(-1, 1)

print("val_data shape: ",val_data.shape)
print("val_labels shape: ",val_labels.shape)


#Test data
test_data = np.array([np.asarray(x) for x in df['Espectrograma'][int(split):]])
test_data = test_data.astype(np.float)
test_data = test_data.reshape(-1, 128, 240, 1)

test_labels = np.asarray(df["Actor"][int(split):])
test_labels = test_labels.astype(np.float)
test_labels = test_labels.reshape(-1, 1)

print("test_data shape: ",test_data.shape)
print("test_labels shape: ",test_labels.shape)


# Definimos la arquitectura para el modelo de Caracteristicas de Voz en personas

**Para las caracteristicas de usuario usaremos la siguiente arquitectura**

In [ ]:
import tensorflow as tf

# Definir la arquitectura de la red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 240, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.08)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.04)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='softmax')
])
model.summary()

# Compilación del modelo

In [ ]:
# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo

In [ ]:
# Entrenar el modelo

model.fit(train_data, 
          train_labels, 
          epochs=100, 
          steps_per_epoch = 12,
          validation_steps = 3,
          validation_data=(val_data, val_labels))

# Evaluación del Modelo

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Guardamos el modelo

In [ ]:
model.save('Personas.h5')

In [ ]:
num = 51
array = model.predict(test_data)[num]
print(np.where(array == max(array)), max(array),test_labels[num])
print(np.where(array == max(array))[0])


# Datos para el modelo de Detección de Sentimientos

In [ ]:
porcentaje = 0.8
total = len(df["Emotion"])
split = porcentaje*total


data_c = np.array([np.asarray(x) for x in df['Espectrograma'][:int(split)]])
data_e = np.asarray(df["Emotion"][:int(split)])

porcentaje = 0.8
total = len(data_c)
split2 = porcentaje*total

#Train data 
train_data = data_c[:int(split2)]
train_data = train_data.astype(np.float)
train_data = train_data.reshape(-1, 128, 240, 1)

train_labels = data_e[:int(split2)]
train_labels = np.asarray(train_labels)
train_labels = train_labels.astype(np.float)
train_labels = train_labels.reshape(-1, 1)

print("train_data shape: ",train_data.shape)
print("train_labels shape: ",train_labels.shape)

#Valid data
val_data = data_c[int(split2):]
val_data = val_data.astype(np.float)
val_data = val_data.reshape(-1, 128, 240, 1)

val_labels = data_e[int(split2):]
val_labels = val_labels.astype(np.float)
val_labels = val_labels.reshape(-1, 1)

print("val_data shape: ",val_data.shape)
print("val_labels shape: ",val_labels.shape)


#Test data
test_data = np.array([np.asarray(x) for x in df['Espectrograma'][int(split):]])
test_data = test_data.astype(np.float)
test_data = test_data.reshape(-1, 128, 240, 1)

test_labels = np.asarray(df["Emotion"][int(split):])
test_labels = test_labels.astype(np.float)
test_labels = test_labels.reshape(-1, 1)

print("test_data shape: ",test_data.shape)
print("test_labels shape: ",test_labels.shape)

# Definimos la arquitectura para el modelo de Caracteristicas de emociones

In [ ]:
import tensorflow as tf

# Definir la arquitectura de la red neuronal
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 240, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='softmax')
])
model_2.summary()

# Compilamos el modelo

In [ ]:
# Compilar el modelo
model_2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo

In [ ]:
# Entrenar el modelo

model_2.fit(train_data, 
          train_labels, 
          epochs=150, 
          steps_per_epoch = 12,
          validation_steps = 3,
          validation_data=(val_data, val_labels))

# Evaluacion de Modelo

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model_2.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Guardamos el modelo

In [ ]:
model.save('emociones_Spec.h5')